In [ ]:
#Library required
from google.colab import drive
import pandas as pd
from pathlib import Path
import os
import json
from google.cloud import bigquery
drive.mount('/content/drive')

In [ ]:
# @title
# connecting notebook with gdrive to access the dataset (I have uploaded to my g drive)
#folder_path = '/content/drive/MyDrive/pran-project-470608-2f7bfba6b03c.json'

# Read service account key
folder_path ='' # copy the JSOn key path here to connect with Big query


In [ ]:
#Path to access the dataset to gdrive
folder = Path("/content/drive/MyDrive/Datasets-2025-08-08")
files = sorted(folder.glob("*.jsonl"))

In [ ]:
#Code to Flatten the JSONl data

to_list = lambda x: [] if x is None else (x if isinstance(x, list) else [x])

all_data, inc_company, inc_article = [], [], [] #Diffent list to store diffent set of arrays

# for fp in files:
#     with fp.open() as f:
#         for line in f:
#             obj = json.loads(line)
#             all_data += to_list(obj.get("data"))
#             all_inc  += to_list(obj.get("included"))

for fp in files:
    with fp.open() as f:
        for line in f:
            obj = json.loads(line)

            # grab data as usual
            all_data += to_list(obj.get("data"))

            # split included into 2 buckets
            for inc in to_list(obj.get("included")):
                if inc.get("type") == "company":
                    inc_company.append(inc)
                elif inc.get("type") == "news_article":
                    inc_article.append(inc)


#Normalizing data to get data in row and column format
df_data       = pd.json_normalize(all_data, sep=".")
df_included_1 = pd.json_normalize(inc_company, sep=".")     # companies
df_included_2 = pd.json_normalize(inc_article, sep=".")

#to get size of df we generated
print("data rows:", len(df_data), "| included rows:", len(df_included_1), "| included2 rows:", len(df_included_2))

In [ ]:
#to count the id in case of duplicacy
df_included_1['id'].value_counts()

In [ ]:
df_included_1[df_included_1['id'] == '7b7dbf17-a2ad-54cc-ac66-20995d7f6fba']

In [ ]:
# Dropping the duplicate rows if any
df_included_1 = df_included_1.drop_duplicates()
df_included_1

In [ ]:
df_included_2 = df_included_2.drop_duplicates()
df_included_2

In [ ]:
df_included_1['id'].value_counts()

In [ ]:
df_data.columns.tolist()

In [ ]:
df_data.head(4)

In [ ]:
#Cleaning the column names

df_data.rename(columns = {
    'attributes.summary':'summary',
 'attributes.category':'category',
 'attributes.found_at':'found_date',
 'attributes.confidence':'confidence_score',
 'attributes.article_sentence':'article_sentence',
 'attributes.human_approved':'human_approved',
 'attributes.planning':'planning',
 'attributes.amount':'amount',
 'attributes.amount_normalized':'amount_normalized',
 'attributes.assets':'assets',
 'attributes.assets_tags':'assets_tags',
 'attributes.award':'award',
 'attributes.contact':'contact',
 'attributes.division':'division',
 'attributes.effective_date':'effective_date',
 'attributes.event':'event',
 'attributes.financing_type':'financing_type',
 'attributes.financing_type_normalized':'financing_type_normalized',
 'attributes.financing_type_tags':'financing_type_tags',
 'attributes.headcount':'headcount',
 'attributes.job_title':'job_title',
 'attributes.job_title_tags':'job_title_tags',
 'attributes.location':'location',
 'attributes.location_data':'location_data',
 'attributes.product':'product',
 'attributes.product_data.full_text':'product_full_text',
 'attributes.product_data.name':'product_name',
 'attributes.product_data.release_type':'product_release_type',
 'attributes.product_data.release_version':'product_release_version',
 'attributes.product_data.fuzzy_match':'product_fuzzy_match',
 'attributes.product_tags':'product_tags',
 'attributes.recognition':'recognization',
 'attributes.vulnerability':'vulnerability',
 'relationships.company1.data.id':'relationship_company1_id',
 'relationships.company1.data.type':'relationship_company1_type',
 'relationships.most_relevant_source.data.id':'relationship_most_relevant_id',
 'relationships.most_relevant_source.data.type':'relationship_most_relevant_type',
 'relationships.company2.data.id':'relationship_company2_id',
 'relationships.company2.data.type':'relationship_company2_type'
},inplace=True)

In [ ]:
df_data

In [ ]:
df_company_details = df_included_1
df_company_details.columns.tolist()

In [ ]:
df_company_details.rename(columns = {
    'attributes.domain':'domain',
    'attributes.company_name':'company_name',
 'attributes.ticker':'ticker'
                          },inplace=True)

In [ ]:
df_company_details.head(2)

In [ ]:
df_details = df_included_2
df_details.columns.tolist()

In [ ]:
df_details.rename(columns = {
     'attributes.author':'author',
 'attributes.body':'body',
 'attributes.image_url':'image_url',
 'attributes.url':'url',
 'attributes.published_at':'published_at',
 'attributes.title':'title'
                          },inplace=True)
df_details

In [ ]:
df_data.dtypes

In [ ]:
#correcting the datatype where required
df_data["found_date"] = pd.to_datetime(df_data["found_date"])
df_data["effective_date"] = pd.to_datetime(df_data["effective_date"])
df_data.info()

In [ ]:
df_company_details.info()

In [ ]:
df_details.info()

In [ ]:
df_details["published_at"] = pd.to_datetime(df_details["published_at"])
df_details.info()

In [ ]:
#Connecting big query to Notebook
client = bigquery.Client.from_service_account_json(folder_path)

In [ ]:
#DDL command to create the new table where we will be strong the data

query1 = """
CREATE TABLE firmable.article_details (
    id STRING NOT NULL,
    type STRING,
    author STRING,
    body STRING,
    image_url STRING,
    url STRING,
    published_at TIMESTAMP,
    title STRING
);


"""
result = client.query(query1).to_dataframe()


In [ ]:
query2 = """
CREATE TABLE firmable.company_details (
    id STRING NOT NULL,
    type STRING,
    domain STRING,
    company_name STRING,
    ticker STRING
);

"""
result = client.query(query2).to_dataframe()


In [ ]:
query3 = """
CREATE TABLE firmable.data_table (
    id STRING NOT NULL,
    type STRING,
    summary STRING,
    category STRING,
    found_date TIMESTAMP,
    confidence_score FLOAT64,
    article_sentence STRING,
    human_approved BOOL,
    planning BOOL,
    amount STRING,
    amount_normalized FLOAT64,
    assets STRING,
    assets_tags STRING,
    award STRING,
    contact STRING,
    division STRING,
    effective_date DATE,
    event STRING,
    financing_type STRING,
    financing_type_normalized STRING,
    financing_type_tags STRING,
    headcount FLOAT64,
    job_title STRING,
    job_title_tags STRING,
    location STRING,
    location_data STRING,
    product STRING,
    product_full_text STRING,
    product_name STRING,
    product_release_type STRING,
    product_release_version STRING,
    product_fuzzy_match STRING,
    product_tags STRING,
    recognization STRING,
    vulnerability STRING,
    relationship_company1_id STRING,
    relationship_company1_type STRING,
    relationship_most_relevant_id STRING,
    relationship_most_relevant_type STRING,
    relationship_company2_id STRING,
    relationship_company2_type STRING
);
"""

result = client.query(query3).to_dataframe()

In [ ]:
#Declaring project name and dataset id
project_id = "pran-project-470608"
dataset_id = "firmable"

In [ ]:
# ingestion thr data to database
df_data.to_gbq(destination_table  = f"{dataset_id}.data_quality_error_report",
          project_id = project_id,
          if_exists="append",   # options: 'fail', 'replace', 'append'
          )
print(f"data inserted succesfully to data_table")

In [ ]:
df_company_details.to_gbq(destination_table  = f"{project_id}.{dataset_id}.company_details",
          project_id = project_id,
          if_exists="append",   # options: 'fail', 'replace', 'append'
          )
print(f"data inserted succesfully to company_details")

In [ ]:
df_details.to_gbq(destination_table  = f"{project_id}.{dataset_id}.article_details",
          project_id = project_id,
          if_exists="append",   # options: 'fail', 'replace', 'append'
          )
print(f"data inserted succesfully to article_details")